In [2]:
import openpyxl
import os
import pandas as pd



In [12]:
import pandas as pd

archivo_excel = "BDD-DR-MQM-001-V0.xlsm"
nombre_hoja = "Cu"

df = pd.read_excel(archivo_excel, sheet_name=nombre_hoja, header=None)
datos_compilados = []

# Extraer los nombres de punto de monitoreo (fila 0, desde columna 6)
puntos = df.iloc[0, 6:].values


print(f"Total de puntos encontrados: {len(puntos)}")


# Extraer las fechas y valores (desde fila 2 en adelante)
for i in range(2, len(df)):
    fecha = df.iloc[i, 5]  # Columna 5 es la fecha
    
    # Saltar si la fecha está vacía
    if pd.isna(fecha):
        continue
    
    # Recorrer SOLO las columnas que corresponden a puntos
    # Desde columna 6 hasta columna 6 + número de puntos
    for j in range(len(puntos)):
        columna_actual = 6 + j  # Columna 6 es donde empiezan los datos
        
        # Verificar que la columna existe
        if columna_actual < len(df.columns):
            valor_medicion = df.iloc[i, columna_actual]
        else:
            valor_medicion = None
        
        # Agregar TODOS los registros, incluyendo vacíos
        datos_compilados.append({
            'punto': puntos[j],
            'fecha': fecha,
            'valor': nombre_hoja,
            'medicion': valor_medicion if pd.notna(valor_medicion) else None
        })

# Crear DataFrame final
df_final = pd.DataFrame(datos_compilados)

print(f"\nTotal de registros compilados: {len(df_final)}")
print("\nPrimeros 20 registros:")
print(df_final.head(20))

# Guardar
df_final.to_excel("datos_compilados.xlsx", index=False)
print("\n✓ Archivo guardado")

Total de puntos encontrados: 18

Total de registros compilados: 65322

Primeros 20 registros:
        punto                fecha valor medicion
0          D1  2026-01-22 00:00:00    Cu    14.18
1          D2  2026-01-22 00:00:00    Cu     None
2          D3  2026-01-22 00:00:00    Cu     None
3          D6  2026-01-22 00:00:00    Cu     None
4          D7  2026-01-22 00:00:00    Cu     None
5          D8  2026-01-22 00:00:00    Cu     None
6          D9  2026-01-22 00:00:00    Cu     None
7         D10  2026-01-22 00:00:00    Cu     None
8         D11  2026-01-22 00:00:00    Cu     None
9    TI (D15)  2026-01-22 00:00:00    Cu     None
10   T2 (DRT)  2026-01-22 00:00:00    Cu     None
11   T3 (DRQ)  2026-01-22 00:00:00    Cu    0.378
12          -  2026-01-22 00:00:00    Cu        1
13        D12  2026-01-22 00:00:00    Cu     None
14  D12 Noche  2026-01-22 00:00:00    Cu     None
15  D13 Noche  2026-01-22 00:00:00    Cu     None
16   D2 Noche  2026-01-22 00:00:00    Cu     None
17    

In [14]:
import pandas as pd

archivo_excel = "BDD-DR-MQM-001-V0.xlsm"

# Define aquí las hojas que quieres procesar
hojas_a_procesar = ["Cu", "Zn", "Fe", "Al"]  # Cambia estos nombres según tus hojas

def procesar_hoja(archivo_excel, nombre_hoja):
    """
    Procesa una hoja específica y retorna el DataFrame compilado
    """
    print(f"\n{'='*60}")
    print(f"PROCESANDO HOJA: {nombre_hoja}")
    print(f"{'='*60}")
    
    try:
        df = pd.read_excel(archivo_excel, sheet_name=nombre_hoja, header=None)
        datos_compilados = []
        
        # Extraer los nombres de punto de monitoreo (fila 0, desde columna 6)
        puntos = df.iloc[0, 6:].values
        
        print(f"Total de puntos encontrados: {len(puntos)}")
        
        # Extraer las fechas y valores (desde fila 2 en adelante)
        for i in range(2, len(df)):
            fecha = df.iloc[i, 5]  # Columna 5 es la fecha
            
            # Saltar si la fecha está vacía
            if pd.isna(fecha):
                continue
            
            # Recorrer SOLO las columnas que corresponden a puntos
            for j in range(len(puntos)):
                columna_actual = 6 + j
                
                # Verificar que la columna existe
                if columna_actual < len(df.columns):
                    valor_medicion = df.iloc[i, columna_actual]
                else:
                    valor_medicion = None
                
                # Agregar TODOS los registros, incluyendo vacíos
                datos_compilados.append({
                    'punto': puntos[j],
                    'fecha': fecha,
                    'metal': nombre_hoja,
                    'medicion': valor_medicion if pd.notna(valor_medicion) else None
                })
        
        # Crear DataFrame final
        df_final = pd.DataFrame(datos_compilados)
        
        print(f"Total de registros compilados: {len(df_final)}")
        print(f"Registros con medición: {df_final['medicion'].notna().sum()}")
        print(f"Registros sin medición: {df_final['medicion'].isna().sum()}")
        
        return df_final
    
    except Exception as e:
        print(f"✗ Error al procesar hoja '{nombre_hoja}': {e}")
        return None


# ============================================
# PROCESAR TODAS LAS HOJAS DEFINIDAS
# ============================================

todos_los_datos = []  # Para guardar todos los datos juntos (opcional)

for hoja in hojas_a_procesar:
    df_hoja = procesar_hoja(archivo_excel, hoja)
    
    if df_hoja is not None and not df_hoja.empty:
        # Guardar archivo individual por hoja
        nombre_archivo = f"datos_compilados_{hoja}.xlsx"
        df_hoja.to_excel(nombre_archivo, index=False)
        print(f"✓ Archivo guardado: {nombre_archivo}")
        
        # Agregar a la lista de todos los datos
        todos_los_datos.append(df_hoja)
    else:
        print(f"⚠ No se generó archivo para la hoja '{hoja}'")

# ============================================
# OPCIONAL: GUARDAR TODOS LOS DATOS EN UN SOLO ARCHIVO
# ============================================

if todos_los_datos:
    df_completo = pd.concat(todos_los_datos, ignore_index=True)
    
    print(f"\n{'='*60}")
    print("RESUMEN GENERAL DE TODAS LAS HOJAS")
    print(f"{'='*60}")
    print(f"Total de registros: {len(df_completo)}")
    print(f"Hojas procesadas: {df_completo['metal'].nunique()}")
    print(f"Puntos únicos: {df_completo['punto'].nunique()}")
    print(f"Fechas únicas: {df_completo['fecha'].nunique()}")
    
    print("\nRegistros por hoja:")
    print(df_completo['metal'].value_counts())
    
    # Guardar archivo consolidado
    df_completo.to_excel("datos_compilados_TODOS.xlsx", index=False)
    print(f"\n✓ Archivo consolidado guardado: datos_compilados_TODOS.xlsx")
else:
    print("\n⚠ No se procesaron datos de ninguna hoja")

print("\n" + "="*60)
print("PROCESO COMPLETADO")
print("="*60)


PROCESANDO HOJA: Cu
Total de puntos encontrados: 18
Total de registros compilados: 65322
Registros con medición: 14585
Registros sin medición: 50737
✓ Archivo guardado: datos_compilados_Cu.xlsx

PROCESANDO HOJA: Zn
Total de puntos encontrados: 20
Total de registros compilados: 44240
Registros con medición: 22146
Registros sin medición: 22094
✓ Archivo guardado: datos_compilados_Zn.xlsx

PROCESANDO HOJA: Fe
Total de puntos encontrados: 18
Total de registros compilados: 39870
Registros con medición: 21368
Registros sin medición: 18502
✓ Archivo guardado: datos_compilados_Fe.xlsx

PROCESANDO HOJA: Al
Total de puntos encontrados: 19
Total de registros compilados: 32262
Registros con medición: 14074
Registros sin medición: 18188
✓ Archivo guardado: datos_compilados_Al.xlsx

RESUMEN GENERAL DE TODAS LAS HOJAS
Total de registros: 181694
Hojas procesadas: 4
Puntos únicos: 18
Fechas únicas: 1120

Registros por hoja:
metal
Cu    65322
Zn    44240
Fe    39870
Al    32262
Name: count, dtype: int6

In [19]:
import pandas as pd

archivo_excel = "BDD-DR-MQM-001-V0.xlsm"

# Define aquí las hojas que quieres procesar
hojas_a_procesar = ["Cu", "Zn", "Al"]

# CONFIGURACIÓN DE COLUMNAS
columna_fecha = 5          # Índice de la columna de fecha (F = 5)
columna_inicio_datos = 6   # Índice donde empiezan los datos (G = 6)
columna_fin_datos = 23   # Índice donde terminan los datos (ajusta según necesites)

def procesar_hoja(archivo_excel, nombre_hoja, col_fecha=5, col_inicio=6, col_fin=None):
    """
    Procesa una hoja específica y retorna el DataFrame compilado
    
    Parámetros:
    - col_fecha: índice de la columna de fecha
    - col_inicio: índice de la primera columna de datos
    - col_fin: índice de la última columna de datos (None = hasta el final)
    """
    print(f"\n{'='*60}")
    print(f"PROCESANDO HOJA: {nombre_hoja}")
    print(f"{'='*60}")
    print(f"Columna fecha: {col_fecha} | Datos desde columna {col_inicio} hasta {col_fin if col_fin else 'final'}")
    
    try:
        df = pd.read_excel(archivo_excel, sheet_name=nombre_hoja, header=None)
        datos_compilados = []
        
        # Definir el rango de columnas a leer
        if col_fin is None:
            col_fin = len(df.columns) - 1
        
        # Extraer los nombres de punto de monitoreo (fila 0, en el rango definido)
        puntos = df.iloc[0, col_inicio:col_fin+1].values
        
        print(f"Total de puntos encontrados: {len(puntos)}")
        print(f"Columnas a procesar: {col_fin - col_inicio + 1}")
        
        # Extraer las fechas y valores (desde fila 2 en adelante)
        for i in range(2, len(df)):
            fecha = df.iloc[i, col_fecha]
            
            # Saltar si la fecha está vacía
            if pd.isna(fecha):
                continue
            
            # Recorrer SOLO las columnas en el rango definido
            for j in range(len(puntos)):
                columna_actual = col_inicio + j
                
                # Verificar que la columna está dentro del rango
                if columna_actual <= col_fin and columna_actual < len(df.columns):
                    valor_medicion = df.iloc[i, columna_actual]
                else:
                    valor_medicion = None
                
                # Agregar TODOS los registros, incluyendo vacíos
                datos_compilados.append({
                    'punto': puntos[j],
                    'fecha': fecha,
                    'valor': nombre_hoja,
                    'medicion': valor_medicion if pd.notna(valor_medicion) else None
                })
        
        # Crear DataFrame final
        df_final = pd.DataFrame(datos_compilados)
        
        print(f"Total de registros compilados: {len(df_final)}")
        print(f"Registros con medición: {df_final['medicion'].notna().sum()}")
        print(f"Registros sin medición: {df_final['medicion'].isna().sum()}")
        
        return df_final
    
    except Exception as e:
        print(f"✗ Error al procesar hoja '{nombre_hoja}': {e}")
        return None


# ============================================
# OPCIÓN 1: MISMO RANGO PARA TODAS LAS HOJAS
# ============================================

todos_los_datos = []

for hoja in hojas_a_procesar:
    df_hoja = procesar_hoja(
        archivo_excel, 
        hoja,
        col_fecha=columna_fecha,
        col_inicio=columna_inicio_datos,
        col_fin=columna_fin_datos
    )
    
    if df_hoja is not None and not df_hoja.empty:
        nombre_archivo = f"datos_compilados_{hoja}.xlsx"
        df_hoja.to_excel(nombre_archivo, index=False)
        print(f"✓ Archivo guardado: {nombre_archivo}")
        todos_los_datos.append(df_hoja)
    else:
        print(f"⚠ No se generó archivo para la hoja '{hoja}'")

# ============================================
# GUARDAR ARCHIVO CONSOLIDADO
# ============================================

if todos_los_datos:
    df_completo = pd.concat(todos_los_datos, ignore_index=True)
    
    print(f"\n{'='*60}")
    print("RESUMEN GENERAL DE TODAS LAS HOJAS")
    print(f"{'='*60}")
    print(f"Total de registros: {len(df_completo)}")
    print(f"Hojas procesadas: {df_completo['valor'].nunique()}")
    print(f"Puntos únicos: {df_completo['punto'].nunique()}")
    print(f"Fechas únicas: {df_completo['fecha'].nunique()}")
    
    print("\nRegistros por hoja:")
    print(df_completo['valor'].value_counts())
    
    df_completo.to_excel("datos_compilados_TODOS.xlsx", index=False)
    print(f"\n✓ Archivo consolidado guardado: datos_compilados_TODOS.xlsx")
else:
    print("\n⚠ No se procesaron datos de ninguna hoja")

print("\n" + "="*60)
print("PROCESO COMPLETADO")
print("="*60)


PROCESANDO HOJA: Cu
Columna fecha: 5 | Datos desde columna 6 hasta 23
Total de puntos encontrados: 18
Columnas a procesar: 18
Total de registros compilados: 65322
Registros con medición: 14585
Registros sin medición: 50737
✓ Archivo guardado: datos_compilados_Cu.xlsx

PROCESANDO HOJA: Zn
Columna fecha: 5 | Datos desde columna 6 hasta 23
Total de puntos encontrados: 18
Columnas a procesar: 18
Total de registros compilados: 39816
Registros con medición: 19944
Registros sin medición: 19872
✓ Archivo guardado: datos_compilados_Zn.xlsx

PROCESANDO HOJA: Al
Columna fecha: 5 | Datos desde columna 6 hasta 23
Total de puntos encontrados: 18
Columnas a procesar: 18
Total de registros compilados: 30564
Registros con medición: 13230
Registros sin medición: 17334
✓ Archivo guardado: datos_compilados_Al.xlsx

RESUMEN GENERAL DE TODAS LAS HOJAS
Total de registros: 135702
Hojas procesadas: 3
Puntos únicos: 18
Fechas únicas: 1119

Registros por hoja:
valor
Cu    65322
Zn    39816
Al    30564
Name: cou